In [2]:
from Bio import Entrez, SeqIO, Medline
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio import Phylo
import requests, sys, json
import re

In [36]:
#filter by identities and coverage - not working when blastp alignment has several hsps
results_Blastp= open('gui.xml')
blastp_records_r = NCBIXML.read(results_Blastp)
first=blastp_records_r.alignments[0]
len_max_seq=first.hsps[0].align_length
len_max_seq

670

In [37]:
results_Blastp= open('gui.xml')
blastp_records = NCBIXML.parse(results_Blastp)
list_accession,list_identities,list_coverage_start, list_coverage_stop,list_alignLen=[],[],[],[],[]

for alignment in  blastp_records:
    for a in alignment.alignments:
        for b in a.hsps:
            list_accession.append(a.accession)
            list_coverage_start.append(b.sbjct_start)
            list_coverage_stop.append(b.sbjct_end)
            list_identities.append(b.identities)
            list_alignLen.append(b.align_length)
            #print(a.accession, b.align_length, b.identities, b.expect, b.sbjct_start, b.sbjct_end)

#calculation of %identity                        
def identity(list_i, list_a):
    perc_list_identities=[]
    for index in range(len(list_i)):
        perc_list_identities.append(round(list_i[index]*100/list_a[index]))
    return perc_list_identities

#calculation of the %coverage - for all hsps of all accessions
def coverage(list_end,list_start):
    perc_list_coverage=[]
    for index in range(len(list_identities)):
        perc_list_coverage.append(round((1+list_end[index]-list_start[index])/len_max_seq*100))
    return perc_list_coverage

#selecting the max coverage and max identity per hsps/accession
def max_values_by_id(ids, covs, idents): 
    unique_ids = []
    max_coverage = []
    max_identity = []
    for id, cov, ident in zip(ids, covs,idents):
        if id in unique_ids:
            idx = unique_ids.index(id)
            max_coverage[idx] = max(max_coverage[idx], cov)
            max_identity[idx] = max(max_identity[idx], ident)
        else:
            unique_ids.append(id)
            max_coverage.append(cov)
            max_identity.append(ident)
    return unique_ids, max_coverage, max_identity

beta_max=max_values_by_id(list_accession, coverage(list_coverage_stop, list_coverage_start),identity(list_identities,list_alignLen))


In [26]:
# print(beta_max)

(['Q9NVP1', 'Q8K363', 'Q9VD51', 'Q1EA54', 'A4R8B5', 'A1CIQ5', 'Q5BBY1', 'Q4IEK8', 'A1CW14', 'Q4WQM4', 'A2Q9T6', 'Q0UR48', 'Q54S03', 'A3LNR6', 'Q2UUN6', 'Q7S2N9', 'Q09916', 'Q03532', 'Q6CXB7', 'Q6BH93', 'Q6FIL3', 'Q74Z73', 'P0CQ84', 'A5E2Z9', 'A5DID7', 'Q6C7D2', 'Q4P6N0', 'Q0CZS8', 'Q5AK59', 'Q2GMX1', 'Q84T03', 'Q9LIH9', 'Q9SB89', 'Q0DBS1', 'Q13206', 'Q80Y44', 'Q5ZJF6', 'A1CTZ2', 'Q4P5U4', 'Q1EB31', 'Q0CMM5', 'Q4WM60', 'A1DNF9', 'A7ESL8', 'Q2H2J1', 'Q5BFU1', 'Q0UMB9', 'Q2UHB7', 'A2QS00', 'Q54Q94'], [100, 99, 68, 86, 78, 89, 86, 82, 89, 89, 75, 85, 70, 75, 78, 78, 68, 73, 70, 68, 71, 66, 80, 73, 68, 67, 66, 83, 68, 83, 70, 70, 69, 70, 65, 65, 65, 71, 72, 66, 68, 69, 68, 63, 65, 69, 69, 68, 67, 67], [100, 86, 79, 57, 60, 55, 57, 58, 55, 55, 62, 57, 65, 61, 59, 60, 66, 60, 62, 65, 62, 64, 57, 60, 63, 62, 65, 56, 63, 55, 59, 59, 58, 56, 46, 46, 46, 43, 42, 43, 43, 43, 43, 45, 44, 43, 42, 42, 43, 42])


In [40]:
print('{:>5}    {:>5}    {:>5}'.format('ID','%coverage','%identity'))
thresold_coverage=30
thresold_identity=30

for id, cov, ident in zip(beta_max[0], beta_max[1], beta_max[2]):
#     print('{:>5}{:>9}{:>12}'.format(id,cov, ident))
    if cov>thresold_coverage and ident>thresold_identity:
        print('{:>5}{:>9}{:>12}'.format(id,cov, ident))

   ID    %coverage    %identity
Q9NVP1      100         100
Q8K363       99          86
Q9VD51       68          79
Q1EA54       86          57
A4R8B5       78          60
A1CIQ5       89          55
Q5BBY1       86          57
Q4IEK8       82          58
A1CW14       89          55
Q4WQM4       89          55
A2Q9T6       75          62
Q0UR48       85          57
Q54S03       70          65
A3LNR6       75          61
Q2UUN6       78          59
Q7S2N9       78          60
Q09916       68          66
Q03532       73          60
Q6CXB7       70          62
Q6BH93       68          65
Q6FIL3       71          62
Q74Z73       66          64
P0CQ84       80          57
A5E2Z9       73          60
A5DID7       68          63
Q6C7D2       67          62
Q4P6N0       66          65
Q0CZS8       83          56
Q5AK59       68          63
Q2GMX1       83          55
Q84T03       70          59
Q9LIH9       70          59
Q9SB89       69          58
Q0DBS1       70          56
Q13206       65 